In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pickle
import pandas as pd

transactions = pd.read_csv('../../data/freelancer_tax_deductions.csv')

ModuleNotFoundError: No module named 'tensorflow'

In [11]:
transactions.head()

,transaction_id,user_id,date,amount,currency,category,description,deduction_rate,max_limit,merchant,payment_method,country,tax_deductible
0,5280833a-4da9-416e-92f3-713ae32f63b2,101,2024-10-21,7923.67,EUR,Marketing & Advertising,Google Ads campaign,1.0,10000,Uber,Bank Transfer,Germany,True
1,23d051ef-6839-40f6-bf22-e3b381043aa3,101,2024-09-09,6105.90,EUR,Work Equipment,Ergonomic keyboard and mouse,0.5,5000,Microsoft,Bank Transfer,USA,False
2,7f48d640-6680-4b2f-bae5-a5c827644d72,101,2024-08-17,686.59,USD,Internet & Phone,Mobile data plan for work calls,0.4,2500,Amazon,PayPal,Australia,True
3,1ebba401-ff6a-4112-9703-a06e6a98d851,101,2024-10-10,1095.94,USD,Client Entertainment,Dinner with potential client,0.2,1500,Netflix,Bank Transfer,Canada,True
4,dbcff318-0298-4ed3-9b6f-2d73adbecf50,101,2025-02-15,3664.11,EUR,Internet & Phone,Business internet subscription,0.4,2500,Microsoft,Cash,France,False


In [12]:
transactions.shape

(200, 13)

In [13]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   transaction_id  200 non-null    object 
 1   user_id         200 non-null    int64  
 2   date            200 non-null    object 
 3   amount          200 non-null    float64
 4   currency        200 non-null    object 
 5   category        200 non-null    object 
 6   description     200 non-null    object 
 7   deduction_rate  200 non-null    float64
 8   max_limit       200 non-null    int64  
 9   merchant        200 non-null    object 
 10  payment_method  200 non-null    object 
 11  country         200 non-null    object 
 12  tax_deductible  200 non-null    bool   
dtypes: bool(1), float64(2), int64(2), object(8)
memory usage: 19.1+ KB


In [14]:
transactions.describe()

,user_id,amount,deduction_rate,max_limit
count,200.0,200.00000,200.000000,200.00000
mean,101.0,4489.02755,0.584750,5360.00000
std,0.0,4531.86891,0.310341,3834.89663
min,101.0,24.52000,0.200000,1500.00000
25%,101.0,1448.53000,0.300000,3000.00000
50%,101.0,2907.72000,0.500000,4000.00000
75%,101.0,5955.47250,1.000000,5000.00000
max,101.0,22002.24000,1.000000,15000.00000


In [15]:
transactions.describe(include='object')

,transaction_id,date,currency,category,description,merchant,payment_method,country
count,200,200,200,200,200,200,200,200
unique,200,156,3,11,36,9,5,6
top,5280833a-4da9-416e-92f3-713ae32f63b2,2024-04-28,EUR,Transportation,SEO service for website optimization,Microsoft,Bank Transfer,Canada
freq,1,3,75,24,12,31,44,47


In [16]:
transactions.isnull().sum()

transaction_id    0
user_id           0
date              0
amount            0
currency          0
category          0
description       0
deduction_rate    0
max_limit         0
merchant          0
payment_method    0
country           0
tax_deductible    0
dtype: int64

In [7]:
import tensorflow as tf
import pickle
import numpy as np
import json


# Load Model and Tokenizer
model = tf.keras.models.load_model("tax_classifier.h5")
with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

tokenizer_json = tokenizer.to_json()

with open("tokenizer.json", "w") as json_file:
    json.dump(tokenizer_json, json_file)


In [ ]:
def predict_deductible(description):
    """Predict if a transaction is deductible."""
    sequence = tokenizer.texts_to_sequences([description])
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=10)
    prediction = model.predict(padded_sequence)[0][0]
    return bool(prediction > 0.5)